In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#export
from exp.nb_11 import *

## Serializing the model

In [3]:
path = untar_data(URLs.IMAGEWOOF_160)

In [4]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35, 1.)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [5]:
len(il)

12954

In [5]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [7]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [6]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
  phases = create_phases(pct_start)
  sched_lr = combine_scheds(phases, cos_1cycle_anneal(lr / 10., lr, lr / 1e5))
  sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
  return [ParamScheduler('lr', sched_lr),
          ParamScheduler('mom', sched_mom)]

In [7]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [ ]:
learn.fit(40, cbsched)

In [12]:
st = learn.model.state_dict()

In [13]:
type(st)

collections.OrderedDict

In [14]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [15]:
st['10.bias']

tensor([ 0.0021, -0.0021,  0.0025,  0.0039, -0.0004,  0.0017, -0.0011,  0.0031,
        -0.0003, -0.0030], device='cuda:0')

In [8]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.

In [18]:
torch.save(st, mdl_path/'iw5')

## Pets

In [9]:
pets = untar_data(URLs.PETS)

In [10]:
pets.ls()

[Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images')]

In [11]:
pets_path = pets/'images'

In [12]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [13]:
il

ImageList (7390 items)
[Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/Persian_20.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/miniature_pinscher_169.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/newfoundland_77.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/pomeranian_15.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/pomeranian_193.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/keeshond_155.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/samoyed_67.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/english_cocker_spaniel_110.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/saint_bernard_88.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/Persian_262.jpg')...]
Path: /home/kienmn/.fastai/data/oxford-iiit-pet/images

In [14]:
#export
def random_splitter(fn, p_valid):
  return random.random() < p_valid

In [15]:
random.seed(42)

In [16]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [17]:
sd

SplitData
Train: ImageList (6667 items)
[Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/Persian_20.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/newfoundland_77.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/pomeranian_15.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/pomeranian_193.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/keeshond_155.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/samoyed_67.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/saint_bernard_88.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/shiba_inu_93.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/samoyed_53.jpg'), Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/Persian_131.jpg')...]
Path: /home/kienmn/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[Path('/home/kienmn/.fastai/data/oxford-iiit-pet/images/miniature_pinscher_169.jpg'), Path('/home/kienmn/.fastai/data/o

In [18]:
n = il.items[0].name
n

'Persian_20.jpg'

In [19]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'Persian'

In [20]:
def pet_labeler(fn):
  return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [21]:
proc = CategoryProcessor()

In [22]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [23]:
', '.join(proc.vocab)

'Persian, newfoundland, pomeranian, keeshond, samoyed, saint_bernard, shiba_inu, Abyssinian, german_shorthaired, american_bulldog, miniature_pinscher, pug, great_pyrenees, yorkshire_terrier, scottish_terrier, english_setter, Russian_Blue, basset_hound, japanese_chin, Maine_Coon, american_pit_bull_terrier, havanese, Ragdoll, staffordshire_bull_terrier, english_cocker_spaniel, Birman, beagle, Egyptian_Mau, chihuahua, Bengal, wheaten_terrier, Sphynx, British_Shorthair, Siamese, Bombay, leonberger, boxer'

In [24]:
ll.valid.x.tfms = val_tfms

In [25]:
c_out = len(proc.vocab)

In [26]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [28]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [29]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.470315,0.086996,3.497083,0.091286,00:32
1,3.263520,0.145193,3.612718,0.114799,00:26
2,3.022453,0.206990,3.879045,0.152144,00:26
3,2.684220,0.306885,2.617727,0.307054,00:24
4,2.360256,0.408130,2.428090,0.369295,00:25


/home/kienmn/Workspace/JupyterWorkspace/fastai_part2/exp/nb_09.py:176: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  state['grad_avg'].mul_(mom).add_(state['mom_damp'], p.grad.data)


## Custom head

In [30]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [33]:
st = torch.load(mdl_path/'iw5')

In [34]:
m = learn.model

In [35]:
m.load_state_dict(st)

<All keys matched successfully>

In [36]:
cut = next(i for i, o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [37]:
cut

8

In [38]:
xb, yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [39]:
pred = m_cut(xb)

In [40]:
pred.shape

torch.Size([128, 512, 4, 4])

In [41]:
ni = pred.shape[1]

In [30]:
#export
class AdaptiveConcatPool2d(nn.Module):
  def __init__(self, sz=1):
    super().__init__()
    self.output_size = sz
    self.ap = nn.AdaptiveAvgPool2d(sz)
    self.mp = nn.AdaptiveMaxPool2d(sz)
    
  def forward(self, x):
    return torch.cat([self.ap(x), self.mp(x)], 1)

In [43]:
nh = 40

m_new = nn.Sequential(m_cut,
                      AdaptiveConcatPool2d(),
                      Flatten(),
                      nn.Linear(ni * 2, data.c_out))

In [44]:
learn.model = m_new

In [45]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.299995,0.136643,3.357036,0.152144,00:25
1,3.042311,0.201590,4.481794,0.113416,00:25
2,2.845341,0.264587,3.908716,0.123098,00:25
3,2.465875,0.380381,2.664410,0.313970,00:25
4,2.090360,0.506525,2.196454,0.470263,00:25


## adapt_model and gradual unfreezing

In [28]:
def adapt_model(learn, data):
  cut = next(i for i, o in enumerate(learn.model.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
  m_cut = learn.model[:cut]
  xb, yb = get_batch(data.valid_dl, learn)
  pred = m_cut(xb)
  ni = pred.shape[1]
  m_new = nn.Sequential(m_cut,
                        AdaptiveConcatPool2d(),
                        Flatten(),
                        nn.Linear(ni * 2, data.c_out))
  learn.model = m_new

In [47]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

<All keys matched successfully>

In [ ]:
adapt_model(learn, data)

In [49]:
for p in learn.model[0].parameters():
  p.requires_grad_(False)

In [ ]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

In [51]:
for p in learn.model[0].parameters():
  p.requires_grad_(True)

In [ ]:
learn.fit(5, cbsched, reset_opt=True)

## Batch norm transfer

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

In [54]:
def apply_mod(m, f):
  f(m)
  for l in m.children():
    apply_mod(l, f)
    
def set_grad(m, b):
  if isinstance(m, (nn.Linear, nn.BatchNorm2d)):
    return
  if hasattr(m, 'weight'):
    for p in m.parameters():
      p.requires_grad_(b)

In [55]:
apply_mod(learn.model, partial(set_grad, b=False))

In [ ]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

In [61]:
apply_mod(learn.model, partial(set_grad, b=True))

In [ ]:
learn.fit(5, cbsched, reset_opt=True)

Pytorch already has an `apply` method we can use:

In [60]:
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and param groups

In [27]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [ ]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

In [32]:
def bn_splitter(m):
  def _bn_splitter(l, g1, g2):
    if isinstance(l, nn.BatchNorm2d):
      g2 += l.parameters()
    elif hasattr(l, 'weight'):
      g1 += l.parameters()
    for ll in l.children():
      _bn_splitter(ll, g1, g2)
      
  g1, g2 = [], []
  _bn_splitter(m[0], g1, g2)
  
  g2 += m[1:].parameters()
  return g1, g2

In [42]:
a, b = bn_splitter(learn.model)
m = learn.model

In [43]:
test_eq(len(a) + len(b), len(list(m.parameters())))

In [47]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [48]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o: o for o in Learner.ALL_CBS})

In [50]:
cb_types.after_backward

'after_backward'

In [51]:
#export
class DebugCallback(Callback):
  _order = 999
  def __init__(self, cb_name, f=None):
    self.cb_name = cb_name
    self.f = f
    
  def __call__(self, cb_name):
    if cb_name == self.cb_name:
      if self.f:
        self.f(self.run)
      else:
        set_trace()

In [52]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
  phases = create_phases(pct_start)
  sched_lr = [combine_scheds(phases, cos_1cycle_anneal(lr / 10., lr, lr / 1e5)) for lr in lrs]
  sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
  return [ParamScheduler('lr', sched_lr), ParamScheduler('mom', sched_mom)]

In [53]:
disc_lr_sched = sched_1cycle([0, 3e-2], 0.5)

In [54]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [55]:
def _print_det(o):
  print(len(o.opt.param_groups), o.opt.hypers)
  raise CancelTrainException()
  
learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


/home/kienmn/Workspace/JupyterWorkspace/fastai_part2/exp/nb_09.py:176: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  state['grad_avg'].mul_(mom).add_(state['mom_damp'], p.grad.data)


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [ ]:
learn.fit(5, disc_lr_sched)

## Export

In [56]:
!python3 notebook2script.py 11a_transfer_learning.ipynb

Converted 11a_transfer_learning.ipynb to nb_11a.py
